<a href="https://colab.research.google.com/github/Espenblo/TDT4173-Project/blob/main/Single_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas_datareader as web
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error              
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MAPE


# **Import Data From Yahoo and create dataframe for first stock**


In [ ]:
# Fetch dataset
#TODO: Pick the stock you want to predict
#df = web.DataReader("BPM", data_source="yahoo", start="1997-01-01", end="2020-01-01") #BP/LUKOY/XOM
#df = web.DataReader("INTC", data_source="yahoo", start="1990-01-01", end="2020-01-01") #INTC/IBM/MSFT
df = web.DataReader("F", data_source="yahoo", start="2004-09-26", end="2020-01-01") #F/TM/TTM

#Dropping a colum from dataframe
df = df.drop(['Adj Close'], axis=1)
df.head()

# **Data info and plot for first stock**

In [ ]:
# Display dataset info
df.info()
df.describe()

In [ ]:
# Illustrate closing price data
df['Close'].plot(figsize=(12,8))

In [ ]:
# Illustrate volume data
df['Volume'].plot(figsize=(12,8))

# **Scale and split data for first stock**

In [ ]:
# Round off the data to two decimals, to remove excess noice
df = df.round(2)

# Create the scalers
scaler_high = MinMaxScaler()
scaler_low = MinMaxScaler()
scaler_open = MinMaxScaler()
scaler_close = MinMaxScaler()
scaler_volume = MinMaxScaler()

# Reshape
high_value = df['High'].values
high_value = high_value.reshape(len(high_value), 1)

low_value = df['Low'].values
low_value = low_value.reshape(len(low_value), 1)

open_value = df['Open'].values
open_value = open_value.reshape(len(open_value), 1)

close_value = df['Close'].values
close_value = close_value.reshape(len(close_value), 1)

volume_value = df['Volume'].values
volume_value = volume_value.reshape(len(volume_value), 1)

# Fit scaler to training data
scaler_high.fit(high_value)
scaler_low.fit(low_value)
scaler_open.fit(open_value)
scaler_close.fit(close_value)
scaler_volume.fit(volume_value)

# Perform scaler transformation
scaled_high = scaler_high.transform(high_value)
scaled_low = scaler_low.transform(low_value)
scaled_open = scaler_open.transform(open_value)
scaled_close = scaler_close.transform(close_value)
scaled_volume = scaler_volume.transform(volume_value)

# Combine data to dataframe 
df['scaled_high'] = scaled_high
df['scaled_low'] = scaled_low
df['scaled_open'] = scaled_open
df['scaled_close'] = scaled_close
df['scaled_volume'] = scaled_volume

# Create final dataframe with scaled values
normalizedData = df[["scaled_high", "scaled_low", "scaled_open", "scaled_close", "scaled_volume"]].values

In [ ]:
# Create an input set and a test set. View the "Split" section of the paper to get an illustration
x = []
y = []

for i in range (0,len(normalizedData)-65):
  x.append(normalizedData[i:i+60])
  y.append(normalizedData[i+60:i+65, 3])

In [ ]:
# Remove values which can't be predicted 5 days ahead
cropped_df = df.tail(-60).head(-5)

# Split into training set (80%) and test set (20%)
split_off_index = round(len(cropped_df)*0.80)

# Split into test set and training set (for evaluation)
train_eval = cropped_df.iloc[:split_off_index]
test_eval = cropped_df.iloc[split_off_index:]

# Split into training set and test set (for training)
train_x = x[:split_off_index]
train_y = y[:split_off_index]

test_x = x[split_off_index:]
test_y = y[split_off_index:]

# Convert to numpy array
train_x = np.array(train_x)
train_y = np.array(train_y)

test_x = np.array(test_x)
test_y = np.array(test_y)


In [ ]:
# Plot Scaled prices
plt.figure(figsize=(16,8))
plt.title('Scaled closing price')
plt.xlabel('Time', fontsize=18)
plt.ylabel('Closing price scaled', fontsize=18)
plt.plot(normalizedData[:,3])
plt.show()


df['Close'].plot(figsize=(12,8))

plt.figure(figsize=(16,8))

plt.xlabel('Time', fontsize=18)
plt.ylabel('Closing price', fontsize=18)
plt.plot(df.Close)
plt.show()

plt.figure(figsize=(16,8))

plt.xlabel('Time', fontsize=18)
plt.ylabel('volume scaled',fontsize =18)
plt.plot(normalizedData[:,4])
plt.show()

plt.figure(figsize=(16,8))

plt.xlabel('Time', fontsize=18)
plt.ylabel('volume 1e7',fontsize =18)
plt.plot(df.Volume)
plt.show()

In [ ]:
# Display dimensionality on training and test sets
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

# **Create the models**

In [ ]:
# Define the model
model_1_layer = Sequential()
model_3_layer = Sequential()

# Add LSTM layer for 1 layer model
model_1_layer.add(LSTM(128, input_shape=(train_x.shape[1], train_x.shape[2])))
model_1_layer.add(Dropout(0.2))


# Add LSTM layer for 3 layer model
model_3_layer.add(LSTM(128, return_sequences=True, input_shape=(train_x.shape[1], train_x.shape[2])))
model_3_layer.add(Dropout(0.2))

model_3_layer.add(LSTM(64, return_sequences=True))
model_3_layer.add(Dropout(0.2))

model_3_layer.add(LSTM(32))
model_3_layer.add(Dropout(0.2))

# Final Prediction - 5 outputs, 5 neurons
model_1_layer.add(Dense(5))
model_3_layer.add(Dense(5))

#Compile
model_1_layer.compile(optimizer='adam', loss='mse')
model_3_layer.compile(optimizer='adam', loss='mse')

In [ ]:
model_1_layer.summary()
model_3_layer.summary()

# **Train individual models on the first stock**

In [ ]:
# Early stop
early_stop = EarlyStopping(monitor='val_loss',patience=20)

# Fit model
def fit_models():
  model_1_layer.fit(train_x, train_y, epochs=200, validation_data=(test_x, test_y), callbacks=[early_stop], batch_size=32)
  model_3_layer.fit(train_x, train_y, epochs=200, validation_data=(test_x, test_y), callbacks=[early_stop,], batch_size=32)

  losses_1_layer = pd.DataFrame(model_1_layer.history.history)
  losses_1_layer.plot()
  losses_3_layer = pd.DataFrame(model_3_layer.history.history)
  losses_3_layer.plot()
fit_models()

# **Let each model make predictions on the test data for the first stock**


In [ ]:
#Create loop and predict
n_features = 5
test_predictions_1_layer = []
test_predictions_3_layer = []


for i in range(0,len(test_x)):
    test_batch = test_x[i]
    reshaped_batch = test_batch.reshape((1, 60, n_features))
    
    # get prediction 1-5 time stamps ahead ([0] is for grabbing just the number instead of [array])
    current_pred_1_layer = model_1_layer.predict(reshaped_batch)[0]
    current_pred_3_layer = model_3_layer.predict(reshaped_batch)[0]
    
    # store prediction
    test_predictions_1_layer.append(current_pred_1_layer)
    test_predictions_3_layer.append(current_pred_3_layer)  

# Inverse transform
true_predictions_1_layer = scaler_close.inverse_transform(test_predictions_1_layer)
true_predictions_3_layer = scaler_close.inverse_transform(test_predictions_3_layer)

# **Compare the actual closing price to the predicted closing price for each model and calculate evaluation metrics (MAPE and RMSE) for each of the 5 predicted days**

In [ ]:
result_array_1_layer = []
result_array_3_layer = []
date_array = []
for i in range(5, len(test_eval)):
  temp_pred_array_1_layer = []
  temp_pred_array_3_layer = []
  date_array.append(test_eval.index[i])
  temp_pred_array_1_layer.append(test_eval['Close'].values[i])
  temp_pred_array_3_layer.append(test_eval['Close'].values[i])
  for j in range(1,6):
    temp_pred_array_1_layer.append(true_predictions_1_layer[i-j][j-1])
    temp_pred_array_3_layer.append(true_predictions_3_layer[i-j][j-1])
  result_array_1_layer.append(temp_pred_array_1_layer)
  result_array_3_layer.append(temp_pred_array_3_layer)


In [ ]:
# Create result dataframes
res_df_1_layer = pd.DataFrame(result_array_1_layer, columns = ['Actual price','1 day prediction',"2 day prediction", "3 day prediction", "4 day prediction", "5 day prediction"])
res_df_1_layer.index = date_array
res_df_3_layer = pd.DataFrame(result_array_3_layer, columns = ['Actual price','1 day prediction',"2 day prediction", "3 day prediction", "4 day prediction", "5 day prediction"])
res_df_3_layer.index = date_array

In [ ]:
# Calculate MAPE and RMSE for 1-5 day predictions
evaluation_array_1_layer = []
evaluation_array_3_layer = []
for i in range (1,6):
  mape_err_1_layer = MAPE(res_df_1_layer.iloc[:,0], res_df_1_layer.iloc[:,i]).numpy().round(4)
  mape_err_3_layer = MAPE(res_df_3_layer.iloc[:,0], res_df_3_layer.iloc[:,i]).numpy().round(4)
  rmse_err_1_layer = np.sqrt(mean_squared_error(res_df_1_layer.iloc[:,0], res_df_1_layer.iloc[:,i])).round(4)
  rmse_err_3_layer = np.sqrt(mean_squared_error(res_df_3_layer.iloc[:,0], res_df_3_layer.iloc[:,i])).round(4)
  evaluation_array_1_layer.append([mape_err_1_layer, rmse_err_1_layer])
  evaluation_array_3_layer.append([mape_err_3_layer, rmse_err_3_layer])

error_df_1_layer = pd.DataFrame(evaluation_array_1_layer, columns = ['MAPE','RMSE'])
error_df_1_layer.index = ['1 day prediction',"2 day prediction", "3 day prediction", "4 day prediction", "5 day prediction"]
print("Single model 1 layer network")
print(error_df_1_layer)
print()
error_df_3_layer = pd.DataFrame(evaluation_array_3_layer, columns = ['MAPE','RMSE'])
error_df_3_layer.index = ['1 day prediction',"2 day prediction", "3 day prediction", "4 day prediction", "5 day prediction"]
print("Single model 3 layer network")
print(error_df_3_layer)

In [ ]:
# Plot actual closing price against predicted closing price for 1 layer model
plt.figure(figsize=(16,8))
plt.title('Actual vs predicted closing price for single 1 layer model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close price USD ($)', fontsize=18)
plt.plot(res_df_1_layer['Actual price'])
plt.plot(res_df_1_layer['1 day prediction'])
plt.plot(res_df_1_layer['5 day prediction'])
plt.xlim(datetime.date(2017,1,1), datetime.date(2020,1,1))
plt.legend(['Actual price', '1 day prediction', "5 day prediction"], loc='lower right')
plt.show()

# Plot actual closing price against predicted closing price for 3 layer model
plt.figure(figsize=(16,8))
plt.title('Actual vs predicted closing price for single 3 layer model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close price USD ($)', fontsize=18)
plt.plot(res_df_3_layer['Actual price'])
plt.plot(res_df_3_layer['1 day prediction'])
plt.plot(res_df_3_layer['5 day prediction'])
plt.xlim(datetime.date(2017,1,1), datetime.date(2020,1,1))
plt.legend(['Actual price', '1 day prediction', "5 day prediction"], loc='lower right')
plt.show()

# **Save single models to google drive (optional)**

In [ ]:
#Save and load model imports
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive 
#from google.colab import auth 
#from oauth2client.client import GoogleCredentials
 
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()                       
#drive = GoogleDrive(gauth)

#Change save names 
# Save models
#model_1_layer.save('Single-1-layer-model-F.h5')
#model_1_layer_file = drive.CreateFile({'title' : 'Single-1-layer-model-F.h5'})                       
#model_1_layer_file.SetContentFile('Single-1-layer-model-F.h5')                       
#model_1_layer_file.Upload()

#model_3_layer.save('Single-3-layer-model-F.h5')
#model_3_layer_file = drive.CreateFile({'title' : 'Single-3-layer-model-F.h5'})                       
#model_3_layer_file.SetContentFile('Single-3-layer-model-F.h5')                       
#model_3_layer_file.Upload()
 
#Get drive ID - MAKE SURE TO SAVE THE ID SOMEWHERE
#print("Single 1 layer model:")              
#print(drive.CreateFile({'id': model_1_layer_file.get('id')}))
#print("Single 3 layer model:")  
#print(drive.CreateFile({'id': model_3_layer_file.get('id')}))


# **Load model**

In [ ]:
#model = load_model("Combined-3-layer-model-F.h5")
#model = model.summary()